In [5]:
import pandas as pd
from erddapy import ERDDAP
import re

In [6]:
# Let's define a tool that look through a server for datasets containing the listed standard variables
# Output is a table that list
# + server
# + dataset ID
# + distinct location (lat/long)
# + standard variables and their corresponding variable name

# Those following information could be added in the future to help filtering the datasets for each distinct location
# + Earliest time attached to location
# + Latest time attached to location
# + Shallowest depth
# + Deepest depth


def get_standard_variables_and_metadata(server_link, standard_variable_list):

    # Get access to the server and find datasets associated with standard_name variable listed
    e = ERDDAP(server=server_link,
               protocol='tabledap',
               response='csv')

    # Define Filter for which datasets to look into
    kw = {'standard_name': ','.join(standard_variable_list),
          'min_lon': -180.0, 'max_lon': 180.0,
          'min_lat': -90.0, 'max_lat': 90.0,
          'min_time': '', 'max_time': '',
          'cdm_data_type': ''}

    # Get available datasets from that server
    search_url = e.get_search_url(response='csv', **kw)
    datasets = pd.read_csv(search_url)

    # Print results
    print(e.server)
    print(str(len(datasets))+" datasets contains "+', '.join(standard_variable_list))

    # Loop through different data sets and create a metadata dataFrame
    df = pd.DataFrame(columns=['Dataset ID'])

    for index, row in datasets.iterrows():
        # Get Info from dataset (mostly min/max lat/long)
        print(row['Dataset ID'])
        info_url = e.get_info_url(dataset_id=row['Dataset ID'], response='csv')
        info = pd.read_csv(info_url)

        # Try to get the distinct lat/long and time and depth range for that dataset, if it fails rely on the
        # ERDDAP metadata
        try:
            # If dataset is spread out geographically find distinct locations (may not work well for trajectory data)
            latlong_url = e.get_download_url(dataset_id=row['Dataset ID'],
                                             protocol='tabledap',
                                             variables=['latitude', 'longitude', 'time'])

            # Get add to the url commands to get distinct values and ordered with min and max time for each lat/long
            distinctMinMaxTime_url = latlong_url+'&distinct()&orderByMinMax(%22latitude%2Clongitude%2Ctime%22)'

            # Get lat/long and min/max depth for this dataset
            data = pd.read_csv(distinctMinMaxTime_url, header=[0, 1])

            # Group data by latitude/longitude and get min max values
            data_reduced = data.groupby(by=[('latitude', 'degrees_north'),
                                            ('longitude', 'degrees_east')]).agg(['min', 'max'])

            if info[(info['Variable Name'] == 'depth')].size > 0:
                latlongdepth_url = e.get_download_url(dataset_id=row['Dataset ID'],
                                                      protocol='tabledap',
                                                      variables=['latitude', 'longitude', 'depth'])

                # Get add to the url commands to get distinct values and ordered with min and max depth for
                # each lat/long
                distinctMinMaxDepth_url = latlongdepth_url + \
                                          '&distinct()&orderByMinMax(%22latitude%2Clongitude%2Cdepth%22)'

                # Get lat/long and min/max depth for this dataset
                data_depth = pd.read_csv(distinctMinMaxDepth_url, header=[0, 1])

                # Group depth data by lat/long and get min max values
                data_depth_reduced = data_depth.groupby(by=[('latitude', 'degrees_north'),
                                                            ('longitude', 'degrees_east')]
                                                        ).agg(['min', 'max'])

                # Merge depth values with time
                data_reduced = data_reduced.merge(data_depth_reduced, left_index=True, right_index=True)

            # Merge multi index column names
            data_reduced.columns = data_reduced.columns.map(' '.join).str.strip(' ')

        except Exception as exception_error:

            print('Failed to read: ' + str(exception_error))
            # If there's only one location, it could get the range from metadata

            # Find lat/long range of this dataset, if it's point we don't need to look into it
            min_latitude = info[info['Attribute Name'] == 'geospatial_lat_min']['Value'].astype('double').values
            max_latitude = info[info['Attribute Name'] == 'geospatial_lat_max']['Value'].astype('double').values
            min_longitude = info[info['Attribute Name'] == 'geospatial_lon_min']['Value'].astype('double').values
            max_longitude = info[info['Attribute Name'] == 'geospatial_lon_max']['Value'].astype('double').values

            # If min/max lat/long are the same don't go in the dataset
            if (min_latitude == max_latitude) & (min_longitude == max_longitude):
                data_reduced = pd.DataFrame(columns=['Dataset ID'])
                data_reduced['latitude degrees_north'] = min_latitude
                data_reduced['longitude degrees_east'] = min_longitude

                if info[(info['Variable Name'] == 'depth')].size > 0:
                    data_reduced['depth m min'] = info[info['Attribute Name'] == 'geospatial_lon_max']['Value'].astype(
                        'double').values

                data_reduced = data_reduced.set_index(['latitude degrees_north', 'longitude degrees_east'])

                print('Retrieved metadata')
            else:
                # Won't handle data with multiple location that it can't retrieve the data
                continue

        # Add Standard Name Variable Name to table info['Attribute Name'] == 'geospatial_lat_min'
        for var in standard_variable_list:
            data_reduced[var] = ','.join(e.get_var_by_attr(dataset_id=row['Dataset ID'],  standard_name=var))

        # Add cdm_data_type to table
        data_reduced['cdm_data_type'] = ','.join(info[info['Attribute Name'] == 'cdm_data_type']['Value'].values)

        # Add Dataset id to the table
        data_reduced['Dataset ID'] = row['Dataset ID']

        # Merge that dataset ID with previously downloaded data
        df = df.append(data_reduced)

    # Add server to dataframe
    df['server'] = e.server

    # Save resulting dataframe to a CSV, file name is based on the server address
    file_name = re.sub('https*://', '', e.server)
    file_name = re.sub("[\./]", '_', file_name)
    file_name = 'Server_List_'+file_name+'.csv'

    print('Save result to '+file_name)
    df.to_csv(file_name)

    return df

In [6]:
#ERDDAP Links from CIOOS Pacific
cioos_pacific_servers = ['https://catalogue.hakai.org/erddap',
                        'http://dap.onc.uvic.ca/erddap',
                        'https://data.cioospacific.ca/erddap']

standard_variable_list = ['sea_water_practical_salinity','sea_water_temperature']

# Get metadata 
for server in cioos_pacific_servers:
    get_standard_variables_and_metadata(server,standard_variable_list)

https://catalogue.hakai.org/erddap
20 datasets contains sea_water_practical_salinity, sea_water_temperature
HakaiSewardBoL5min


C:\Users\jessy\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


HakaiKetchikanBoL5min
HakaiKodiakBoL5min
HakaiCTDDFO2
HakaiCTDFZH01
HakaiCTDKC10
HakaiCTDPRUTH
HakaiCTDQCS01
HakaiCTDQU20
HakaiCTDQU29
HakaiCTDQU34
HakaiCTDQU38
HakaiCTDQU39
HakaiCTDQU43
HakaiBaynesSoundBoL5min
HakaiQuadraBoL5min
HakaiQuadraLimpet5min
HakaiQuadraWeather5min
HakaiKCBuoy1hour
HakaiSitkaBoL5min
Save result to Server_List_catalogue_hakai_org_erddap.csv
http://dap.onc.uvic.ca/erddap
913 datasets contains sea_water_practical_salinity, sea_water_temperature
scalar_118170
currents_118170
scalar_118895
currents_118895
scalar_1189222
currents_1189222
scalar_1190536
currents_1190536
scalar_1194368
currents_1194368
scalar_1189655
scalar_1190542
scalar_1192794
scalar_1194359
scalar_1200145
scalar_1194361
scalar_1200147
scalar_1189656
scalar_1190543
scalar_1192795
scalar_1194360
scalar_1200146
scalar_1194087
scalar_1194133
scalar_1196717
scalar_1200697
scalar_1194132
scalar_1196714
scalar_1194138
scalar_1196720
scalar_1189272
scalar_1192789
scalar_1200624
scalar_1192791
scalar_11892

scalar_119183
scalar_1189063
scalar_119182
scalar_1189062
scalar_118600
scalar_119123
scalar_118599
scalar_119122
scalar_1191057
scalar_1191058
profile_119268
profile_1190697
profile_1191669
profile_1193040
profile_1193645
profile_1193644
profile_1195373
profile_1195662
scalar_1197239
profile_1190701
profile_1191673
profile_1193045
profile_1193044
profile_1195378
profile_1195666
profile_1195664
profile_119271
profile_1190700
profile_1191672
profile_1193043
profile_1195376
profile_1190698
profile_1191670
profile_1193041
profile_1195374
profile_1195663
scalar_1138
scalar_22656
scalar_1563
scalar_30164
scalar_117484
scalar_118572
scalar_119127
scalar_22651
scalar_1523
scalar_1559
scalar_30162
scalar_117480
scalar_1133
scalar_117572
scalar_117887
scalar_118571
scalar_119126
scalar_1189067
scalar_1189354
scalar_117582
scalar_117888
scalar_1189068
scalar_1189355
scalar_119128
scalar_1189189
scalar_1189356
scalar_119358
scalar_1189944
scalar_1190043
scalar_119355
scalar_1189533
scalar_1189943